In [2]:
import requests
import pandas as pd
from io import StringIO

In [77]:
# Primero entenderme con los csv y ya cuando tal alomejor me pongo con json para traer los datos en inglés

url = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000013/~latest"
params = {"locale": "en"}
headers = {"Accept": "application/json"}

response = requests.get(url, params=params, headers=headers)
data = response.json()

### Variable Descriptions

Description of variable MOVIMIENTO_AERONAVE:

1. Otros servicios comerciales (Other commercial services)

This includes air operations that are commercial, but are not regular passenger or cargo transportation. Examples:

Sightseeing flights (aerial tours)

Crop spraying or aerial work

Aerial advertising (planes with banners)

Aerial photography

Air taxi (though sometimes it falls under “Non-regular”)

In other words, these are services that generate revenue but are not scheduled airline or traditional charter operations.

2. No regular (Non-regular)

Also called charter flights. These are commercial passenger or cargo flights that do not follow a fixed schedule or a regular frequency. Examples:

A flight hired by a group for an event

A humanitarian flight (if commercial)

On-demand operations

3. Regular (regular)

Flights that are scheduled commercial services, with established frequency, fixed routes, and open sale to the public. Examples:

Traditional airlines (LATAM, Iberia, etc.)

Daily/weekly flights with published timetables

4. Comercial (comercial)

This is a generic term that usually encompasses all flights that are for profit (i.e., passenger, cargo, or mail transportation for payment). However, in some datasets, it’s used as a base category when no further detail is provided. It may include both regular and non-regular flights.

In our case is the sum of Regular and Non-Regular

Ignore:
ESTADO_OBSERVACION, CONFIDENCIALIDAD_OBSERVACION

## Tráfico por islas y principales territorios de escala o de origen/destino

### 01 Tráfico de pasajeros (Passenger Traffic)

Tráfico de pasajeros registrados en los aeropuertos de las islas de Canarias según movimientos (llegadas y salidas), servicios comerciales y principales territorios de escala (territorio de despegue anterior al territorio de llegada independientemente del origen del vuelo / territorio de aterrizaje siguiente al territorio de salida independientemente del destino del vuelo). Datos mensuales y anuales para Canarias desde el año 2004.

Número de personas embarcadas y desembarcadas (entradas y salidas) de una aeronave en un aeropuerto, excluyendo a los miembros en servicio de las tripulaciones de vuelo y cabina.

For each island (TERRITORIO), for each destination/origin airport (AEROPUERTO_ESCALA), for each type of flight (MOVIMIENTO_AERONAVE), for each month provides the value of the passengers arrived from that airport (MOVIMIENTO_AERONAVE#es: Llegada), heading to that airport (MOVIMIENTO_AERONAVE#es: Salida) and Total \

In [66]:
# URL of the CSV API
url = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000013/~latest.csv?locale=en"

In [67]:
# Send HTTP GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Read CSV data using pandas
    csv_data = StringIO(response.content.decode("utf-8"))
    data = pd.read_csv(csv_data)
else:
    print("Failed to retrieve data. Status code:", response.status_code)

In [68]:
data.columns

Index(['MEDIDAS#es', 'MEDIDAS_CODE', 'TERRITORIO#es', 'TERRITORIO_CODE',
       'AEROPUERTO_ESCALA#es', 'AEROPUERTO_ESCALA_CODE',
       'MOVIMIENTO_AERONAVE#es', 'MOVIMIENTO_AERONAVE_CODE',
       'SERVICIO_AEREO#es', 'SERVICIO_AEREO_CODE', 'TIME_PERIOD#es',
       'TIME_PERIOD_CODE', 'OBS_VALUE', 'ESTADO_OBSERVACION#es',
       'ESTADO_OBSERVACION_CODE', 'CONFIDENCIALIDAD_OBSERVACION#es'],
      dtype='object')

In [69]:
df_passengers_only = data.copy(deep=True)

In [79]:
df_passengers_only.shape

(215040, 8)

In [71]:
df_passengers_only = df_passengers_only.drop(columns=df_passengers_only.columns[df_passengers_only.columns.str.endswith('_CODE')])

In [72]:
# Contains one value for every row (Pasajeros)
df_passengers_only.drop('MEDIDAS#es', axis=1, inplace=True)

In [73]:
df_passengers_only.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215040 entries, 0 to 215039
Data columns (total 8 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   TERRITORIO#es                    215040 non-null  object 
 1   AEROPUERTO_ESCALA#es             215040 non-null  object 
 2   MOVIMIENTO_AERONAVE#es           215040 non-null  object 
 3   SERVICIO_AEREO#es                215040 non-null  object 
 4   TIME_PERIOD#es                   215040 non-null  object 
 5   OBS_VALUE                        143567 non-null  float64
 6   ESTADO_OBSERVACION#es            143567 non-null  object 
 7   CONFIDENCIALIDAD_OBSERVACION#es  0 non-null       float64
dtypes: float64(2), object(6)
memory usage: 13.1+ MB


In [74]:
for c in df_passengers_only.select_dtypes("object").columns:
    if c == "TIME_PERIOD#es":
        continue
    print(c)
    print(df_passengers_only[c].unique())

TERRITORIO#es
['Canarias' 'Lanzarote' 'Fuerteventura' 'Gran Canaria' 'Tenerife'
 'La Gomera' 'La Palma' 'El Hierro']
AEROPUERTO_ESCALA#es
['Reino Unido' 'Extranjero' 'Canarias'
 'Extranjeros y España (excluida Canarias)' 'España (excluida Canarias)'
 'España' 'Total' 'Alemania']
MOVIMIENTO_AERONAVE#es
['Llegada' 'Salida' 'Total']
SERVICIO_AEREO#es
['Comercial' 'Otros servicios comerciales' 'No regular' 'Regular']
ESTADO_OBSERVACION#es
['Valor normal' 'Valor provisional' nan]


In [80]:
df_passengers_only

,TERRITORIO#es,AEROPUERTO_ESCALA#es,MOVIMIENTO_AERONAVE#es,SERVICIO_AEREO#es,TIME_PERIOD#es,OBS_VALUE,ESTADO_OBSERVACION#es,CONFIDENCIALIDAD_OBSERVACION#es
0,Canarias,Reino Unido,Llegada,Comercial,01/2004,317378.0,Valor normal,NaN
1,Canarias,Reino Unido,Llegada,Comercial,02/2004,320960.0,Valor normal,NaN
2,Canarias,Reino Unido,Llegada,Comercial,06/2010,251362.0,Valor normal,NaN
3,Canarias,Reino Unido,Llegada,Comercial,07/2010,296367.0,Valor normal,NaN
4,Canarias,Reino Unido,Llegada,Comercial,08/2010,287040.0,Valor normal,NaN
...,...,...,...,...,...,...,...,...
215035,El Hierro,Alemania,Total,Regular,01/2010,NaN,NaN,NaN
215036,El Hierro,Alemania,Total,Regular,02/2010,NaN,NaN,NaN
215037,El Hierro,Alemania,Total,Regular,03/2010,NaN,NaN,NaN
215038,El Hierro,Alemania,Total,Regular,04/2010,NaN,NaN,NaN


In [ ]:
#df_passengers_only.loc[(df_passengers_only['TIME_PERIOD#es'] == "07/2025") & (df_passengers_only['SERVICIO_AEREO#es'] == "Comercial")].to_csv('borrar.csv', index=False)
#df_passengers_only.loc[(df_passengers_only['TERRITORIO#es'] == "Canarias") & (df_passengers_only['TIME_PERIOD#es'] == "07/2025") & (df_passengers_only["AEROPUERTO_ESCALA#es"] == "Reino Unido") & (df_passengers_only["MOVIMIENTO_AERONAVE#es"] == "Total")].to_csv('borrar.csv', index=False)

### 02 Tráfico de Mercancías y correos (Goods and Mail Traffic)

Tráfico de mercancías y correos registrados en los aeropuertos de las islas de Canarias según movimientos (llegadas y salidas), servicios comerciales y principales territorios de escala (territorio de despegue anterior al territorio de llegada independientemente del origen del vuelo / territorio de aterrizaje siguiente al territorio de salida independientemente del destino del vuelo). Datos mensuales y anuales para Canarias desde el año 2004.

Goods
Peso del total de mercancía transportada, expresado en kilogramos. Donde mercancía es cualquier propiedad transportada en una aeronave que no sea correo, suministros ni equipaje.
Mail
Peso del total de correo transportado, expresado en kilogramos. Donde correo es el envío de correspondencia y otros objetos transportados en una aeronave, que han sido enviados y destinados a ser entregados a las administraciones postales.

In [37]:
# URL of the CSV API
url = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000014/~latest.csv"

In [38]:
# Send HTTP GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Read CSV data using pandas
    csv_data = StringIO(response.content.decode("utf-8"))
    data = pd.read_csv(csv_data)
else:
    print("Failed to retrieve data. Status code:", response.status_code)

In [39]:
df_gm_only = data.copy(deep=True)

In [42]:
df_gm_only.drop(columns=df_gm_only.columns[df_gm_only.columns.str.endswith('_CODE')], inplace=True)

In [81]:
df_gm_only

,MEDIDAS#es,TERRITORIO#es,AEROPUERTO_ESCALA#es,MOVIMIENTO_AERONAVE#es,SERVICIO_AEREO#es,TIME_PERIOD#es,OBS_VALUE,ESTADO_OBSERVACION#es,CONFIDENCIALIDAD_OBSERVACION#es
0,Correo,Canarias,Reino Unido,Llegada,Comercial,01/2004,NaN,NaN,NaN
1,Correo,Canarias,Reino Unido,Llegada,Comercial,02/2004,NaN,NaN,NaN
2,Correo,Canarias,Reino Unido,Llegada,Comercial,03/2004,40.0,Valor normal,NaN
3,Correo,Canarias,Reino Unido,Llegada,Comercial,04/2004,NaN,NaN,NaN
4,Correo,Canarias,Reino Unido,Llegada,Comercial,05/2004,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
430075,Mercancía,El Hierro,Alemania,Total,Regular,2019,NaN,NaN,NaN
430076,Mercancía,El Hierro,Alemania,Total,Regular,01/2020,NaN,NaN,NaN
430077,Mercancía,El Hierro,Alemania,Total,Regular,02/2020,NaN,NaN,NaN
430078,Mercancía,El Hierro,Alemania,Total,Regular,03/2020,NaN,NaN,NaN


In [ ]:
#df_gm_only.to_csv("borrar.csv", index=False)

### 03 Tráfico de Operaciones

Tráfico de operaciones registradas en los aeropuertos de las islas de Canarias según movimientos (llegadas y salidas), servicios comerciales y principales territorios de origen/destino (territorio de origen o destino independientemente de si ha hecho o hará escala en otro territorio). Datos mensuales y anuales para Canarias desde el año 2004.

Operations
Número total de operaciones de aterrizaje y despegue efectuados por las aeronaves en el aeropuerto.

In [47]:
url = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000015/~latest.csv"

In [48]:
# Send HTTP GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Read CSV data using pandas
    csv_data = StringIO(response.content.decode("utf-8"))
    data = pd.read_csv(csv_data)
else:
    print("Failed to retrieve data. Status code:", response.status_code)

In [49]:
df_op_only = data.copy(deep=True)

In [50]:
df_op_only.drop(columns=df_op_only.columns[df_op_only.columns.str.endswith('_CODE')], inplace=True)

In [82]:
df_op_only

,MOVIMIENTO_AERONAVE#es,MEDIDAS#es,AEROPUERTO_ORIGEN_DESTINO#es,SERVICIO_AEREO#es,TIME_PERIOD#es,TERRITORIO#es,OBS_VALUE,ESTADO_OBSERVACION#es,CONFIDENCIALIDAD_OBSERVACION#es
0,Total,Operaciones,Reino Unido,Regular,01/2004,Canarias,425.0,Valor normal,NaN
1,Total,Operaciones,Reino Unido,Regular,02/2004,Canarias,378.0,Valor normal,NaN
2,Total,Operaciones,Reino Unido,Regular,03/2004,Canarias,358.0,Valor normal,NaN
3,Total,Operaciones,Reino Unido,Regular,04/2004,Canarias,280.0,Valor normal,NaN
4,Total,Operaciones,Reino Unido,Regular,05/2004,Canarias,187.0,Valor normal,NaN
...,...,...,...,...,...,...,...,...,...
215035,Llegada,Operaciones,Alemania,Comercial,04/2023,El Hierro,NaN,NaN,NaN
215036,Llegada,Operaciones,Alemania,Comercial,05/2023,El Hierro,NaN,NaN,NaN
215037,Llegada,Operaciones,Alemania,Comercial,06/2023,El Hierro,NaN,NaN,NaN
215038,Llegada,Operaciones,Alemania,Comercial,07/2023,El Hierro,NaN,NaN,NaN


In [ ]:
#df_op_only.to_csv("borrar.csv", index=False)

## Tráfico según tipos de servicios comerciales por islas y principales territorios de escala o de origen/destino

### 04 Tráfico de pasajeros, mercancías, correos y operaciones

Datasets 1, 2 and 3 put together in one

## Tráfico según tipos de servicios comerciales detallados por aeropuertos

### 05 Tráfico total de pasajeros, mercancías, correos y operaciones

Como el dataset 04 pero te pone cada aeropuerto de origen/destino y el total de MOVIMIENTO_AERONAVE, es decir llegadas y salidas



In [64]:
url_passengers = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000001/~latest.csv"
url_gm = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000002/~latest.csv"
url_op = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000003/~latest.csv"

In [65]:
# Send HTTP GET request
response = requests.get(url_passengers)

# Check if the request was successful
if response.status_code == 200:
    # Read CSV data using pandas
    csv_data = StringIO(response.content.decode("utf-8"))
    data = pd.read_csv(csv_data)
else:
    print("Failed to retrieve data. Status code:", response.status_code)

In [54]:
df_all = data.copy(deep=True)

In [58]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2963520 entries, 0 to 2963519
Data columns (total 9 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   SERVICIO_AEREO#es                object 
 1   MEDIDAS#es                       object 
 2   MOVIMIENTO_AERONAVE#es           object 
 3   TIME_PERIOD#es                   object 
 4   AEROPUERTO_BASE#es               object 
 5   AEROPUERTO_ESCALA#es             object 
 6   OBS_VALUE                        float64
 7   ESTADO_OBSERVACION#es            object 
 8   CONFIDENCIALIDAD_OBSERVACION#es  float64
dtypes: float64(2), object(7)
memory usage: 203.5+ MB


In [57]:
df_all.drop(columns=df_all.columns[df_all.columns.str.endswith('_CODE')], inplace=True)

In [61]:
df_all.loc[(df_all['MEDIDAS#es'] != "Pasajeros")]

,SERVICIO_AEREO#es,MEDIDAS#es,MOVIMIENTO_AERONAVE#es,TIME_PERIOD#es,AEROPUERTO_BASE#es,AEROPUERTO_ESCALA#es,OBS_VALUE,ESTADO_OBSERVACION#es,CONFIDENCIALIDAD_OBSERVACION#es


In [60]:
df_all.loc[(df_all['TIME_PERIOD#es'] == "06/2025")].to_csv("borrar.csv", index=False)

### 06 Entrada de pasajeros, mercancías y correos y operaciones de aterrizaje

Como el dataset 05 pero solo entrada

### 07 Salida de pasajeros, mercancías y correos y operaciones de despegue

Como el dataset 05 pero solo salida